In [38]:
import pandas as pd
import random

# Task1

In [39]:
attendace_raw = []
cohort_cat = ["alpha", "beta", "gamma"]
for i in range(1,25):
    record = {
        "student_id": f"S{i:03}",
        "cohort": random.choice(cohort_cat),
        "attended_sessions": random.randint(0,7),
        "expected_sessions": 6
    }
    attendace_raw.append(record)
attendance = pd.DataFrame(attendace_raw)

attendance.head() ,attendance.info()

<class 'pandas.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   student_id         24 non-null     str  
 1   cohort             24 non-null     str  
 2   attended_sessions  24 non-null     int64
 3   expected_sessions  24 non-null     int64
dtypes: int64(2), str(2)
memory usage: 900.0 bytes


(  student_id cohort  attended_sessions  expected_sessions
 0       S001   beta                  3                  6
 1       S002  gamma                  5                  6
 2       S003   beta                  6                  6
 3       S004  gamma                  2                  6
 4       S005   beta                  4                  6,
 None)

# Task2

In [40]:
attendance_indexed = attendance.set_index("student_id")
excused_absences = pd.Series({
    'S001': 1, 'S002': 2, 'S013': 1, 'S022': 6, 'S005': 2,
    'S100': 1, 'S101': 1, 'S102': 1, 'S103': 1, 'S104': 1
})
attendance_indexed["adjusted_attendance"] = attendance_indexed["attended_sessions"] + excused_absences
attendance_indexed.head()

,cohort,attended_sessions,expected_sessions,adjusted_attendance
student_id,,,,
S001,beta,3,6,4.0
S002,gamma,5,6,7.0
S003,beta,6,6,NaN
S004,gamma,2,6,NaN
S005,beta,4,6,6.0


In [41]:
attendance_indexed['adjusted_attendance'] = attendance_indexed['adjusted_attendance'].fillna(
    attendance_indexed['attended_sessions']
)
attendance_indexed

,cohort,attended_sessions,expected_sessions,adjusted_attendance
student_id,,,,
S001,beta,3,6,4.0
S002,gamma,5,6,7.0
S003,beta,6,6,6.0
S004,gamma,2,6,2.0
S005,beta,4,6,6.0
S006,beta,4,6,4.0
S007,gamma,3,6,3.0
S008,alpha,2,6,2.0
S009,gamma,0,6,0.0


# Task3

In [42]:
attendance_indexed.iloc[0, 0] = "  Alpha "
attendance_indexed.iloc[1, 0] = "BETA"
attendance_indexed.iloc[12, 0] = "gamma  "
attendance_indexed.iloc[23, 0] = " alPHa"
attendance_indexed["cohort"].unique()

<StringArray>
['  Alpha ', 'BETA', 'beta', 'gamma', 'alpha', 'gamma  ', ' alPHa']
Length: 7, dtype: str

In [43]:
attendance_indexed['cohort'] = attendance_indexed['cohort'].str.strip().str.lower()
attendance_indexed['cohort'].unique()

<StringArray>
['alpha', 'beta', 'gamma']
Length: 3, dtype: str

# Task4


In [44]:
attendance_mask = attendance_indexed["attended_sessions"] < attendance_indexed["expected_sessions"]
low_attendance = attendance_indexed[attendance_mask]
low_attendance

,cohort,attended_sessions,expected_sessions,adjusted_attendance
student_id,,,,
S001,alpha,3,6,4.0
S002,beta,5,6,7.0
S004,gamma,2,6,2.0
S005,beta,4,6,6.0
S006,beta,4,6,4.0
S007,gamma,3,6,3.0
S008,alpha,2,6,2.0
S009,gamma,0,6,0.0
S010,beta,1,6,1.0


In [45]:
cohort_averages = attendance_indexed.groupby("cohort")["attended_sessions"].mean()
cohort_averages

cohort
alpha    2.833333
beta     3.900000
gamma    1.625000
Name: attended_sessions, dtype: float64

# Task5

In [46]:
attendance_ok = attendance_indexed["attended_sessions"] >= attendance_indexed["expected_sessions"]
low_attendance = low_attendance[attendance_ok]
low_attendance

/tmp/ipykernel_75929/330919557.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  low_attendance = low_attendance[attendance_ok]


,cohort,attended_sessions,expected_sessions,adjusted_attendance
student_id,,,,
